In [3]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import scipy as sp
import pandas as pd

from tqdm import tqdm_notebook as tqdm
from matplotlib import pyplot as plt

SEED = 123
np.random.seed(SEED)

# 1 - Convolution 1x1

<img src="images/conv1x1.png">

1x1 convolution по-другому называют Network In Network, т.к. по сути каждый вектор $1\times1\times n_c^{[l]}$ прогоняется через FC сеть и на выходе получается $1\times1\times n_c^{[l+1]}$ (т.е. $n_c^{[l]} \to n_c^{[l+1]}$).

<img src="images/conv1x1use.png">

С помощью 1x1 conv можно менять $n_c$ или добавлять нелинейности, что позволяет аппроксимировать более сложные функции.

Материалы:
- Для лучшего понимания посмотрите [видео](https://www.youtube.com/watch?v=vcp0XvDAX68&t=10s).
- Статья: [Network In Network](https://arxiv.org/abs/1312.4400)

# 2 - Inception

GoogLeNet – архитектура-победитель ILSVRC 2014 г. Главное нововведение – модуль Inception, который значительно уменьшил количество параметров модели (до 40 млн.). Кроме того, большое количество параметров было сокращено вследствие использования усредняющего пулинга вместо полносвязных слоев в конце классификатора. После этого были предложены измененные, улучшенные версии GoogLeNet.

<img src='images/inception_module.png'>

<img src='images/deeper.jpg'>

Материалы:
- [видео 1](https://www.youtube.com/watch?v=C86ZXvgpejM) 
- [видео 2](https://www.youtube.com/watch?v=KfV8CJh7hE0)
- [статья](https://arxiv.org/abs/1409.4842)

In [2]:
from keras.applications import InceptionV3
inception_model = InceptionV3(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
inception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

# 3 - Residual Networks

*ResNet* (от англ. "Residual Network" – буквально "остаточная сеть") – победитель ILSVRC 2015 г., обладающая 152 слоями. Главное нововведение сети – показать, что механизм *batch normalization* и *skip connection ("short cut")* позволяет строить очень глубокие сети. Авторами было показано, что с использованием подобных техник может быть построена сеть и в 1000 слоев глубиной. Однако было эмпирически найдено и показано, что ResNet в основе принятия своего решения опирается прежде всего на первые слои относительно небольшой глубины (первые $~20-30$ слоев), вместо того, чтобы использовать все слои.

## Residual block (identity)

<img src="images/skip_connection.png">


$$z^{[l+1]} = W^{[l+1]}a^{[l]} + b^{[l+1]}$$

$$a^{[l+1]} = g(z^{[l+1]})$$

$$z^{[l+2]} = W^{[l+2]}a^{[l+1]} + b^{[l+2]}$$

$$Plain:  a^{[l+2]} = g(z^{[l+2]})$$
$$Residual: a^{[l+2]} = g(z^{[l+2]} + a^{[l]})$$


$g$ – activation function (ReLU)

## Почему residual block работает?

Если $W^{[l+2]} = 0, b^{[l+2]} = 0$, то 
$$z^{[l+2]} = 0$$

$$a^{[l+2]} = g(a^{[l]}) = a^{[l]}$$

т.к. g – ReLU и $a \geq 0$

Получается, residual block может легко выучить identity function ( $f(x) = x$ ).

Поэтому добавление такого блока точно не ухудшит performance сети, а, возможно, даже улучшит.

Это позволяет создавать очень глубокие сети (до 1000 слоев).

<img src="images/resnet_vs_plain.png">

Статья: [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

In [5]:
from keras.applications import ResNet50
resnet_model = ResNet50(include_top=True, weights='imagenet', input_shape=(224, 224, 3))
resnet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_4[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_238 (Activation)     (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

# Что еще почитать
* [Обзор различных архитектур](https://towardsdatascience.com/neural-network-architectures-156e5bad51ba)